In [2]:
import lightgbm as lgb
from sklearn.metrics import roc_auc_score
import pandas as pd
import numpy as np

data = "/mnt/d/Data/mangaki-data-challenge/latest/"
train = pd.read_csv(data+'train_0.csv')
train['item_category'] = train.item_category.astype('category')
test = pd.read_csv(data+'test_0.csv')
test['item_category'] = test.item_category.astype('category')

params = {
    'task': 'train',
    'boosting_type': 'rf',
    'objective': 'binary',
    'metric': {'auc'},
    'num_leaves': 450,
    'min_data_in_leaf': 8,
    'feature_fraction': 0.6,
    'bagging_fraction': 0.8,
    'bagging_freq': 1,
    'lambda_l1': 0,
    'min_gain_to_split': 0,
    'verbose': 0
}

In [3]:
params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': {'auc'},
    'num_leaves': 40,
    'min_data_in_leaf': 90,
    'feature_fraction': 0.6,
    'bagging_fraction': 0.96,
    'bagging_freq': 1,
    'lambda_l1': 0,
    'verbose': 0
}

In [4]:
X = lgb.Dataset(train.drop(['user_id', 'work_id', 'rating'], axis=1), train['rating'])
gbdt = lgb.train(params, X, num_boost_round=119, valid_sets=[X], verbose_eval=True)
Y = gbdt.predict(test.drop(['user_id', 'work_id'], axis=1))

[1]	training's auc: 0.759454
[2]	training's auc: 0.786068
[3]	training's auc: 0.802597
[4]	training's auc: 0.817182
[5]	training's auc: 0.824382
[6]	training's auc: 0.828355
[7]	training's auc: 0.831602
[8]	training's auc: 0.834986
[9]	training's auc: 0.839036
[10]	training's auc: 0.843212
[11]	training's auc: 0.846991
[12]	training's auc: 0.850615
[13]	training's auc: 0.853322
[14]	training's auc: 0.856913
[15]	training's auc: 0.86063
[16]	training's auc: 0.863923
[17]	training's auc: 0.867107
[18]	training's auc: 0.869813
[19]	training's auc: 0.871613
[20]	training's auc: 0.874247
[21]	training's auc: 0.876448
[22]	training's auc: 0.878947
[23]	training's auc: 0.881521
[24]	training's auc: 0.884322
[25]	training's auc: 0.887147
[26]	training's auc: 0.889298
[27]	training's auc: 0.891788
[28]	training's auc: 0.893882
[29]	training's auc: 0.895678
[30]	training's auc: 0.897765
[31]	training's auc: 0.899408
[32]	training's auc: 0.901417
[33]	training's auc: 0.903078
[34]	training's auc:

In [2]:
# fm training
from scipy.sparse import hstack
from fastFM import sgd
from sklearn.metrics import roc_auc_score
from sklearn.datasets import load_svmlight_file

path = "/mnt/d/Data/mangaki-data-challenge/libfm/"

params = {
    'n_iter':340000, 
    'init_stdev':0.01,
    'l2_reg_w':0.00025,
    'l2_reg_V':0.5,
    'rank':1,
    'step_size':0.02
}

train, ty = load_svmlight_file(path+"train_0.csv", zero_based=True)
test, _ = load_svmlight_file(path+"test_0.csv")
fm = sgd.FMClassification(**params)
fm.fit(train, np.require(ty*2-1, dtype=np.int))
Y = fm.predict_proba(test)

In [5]:
test = pd.read_csv(data+'../test.csv', dtype={
    'user_id': np.int16,
    'work_id': np.int16
})
#test['prob_willsee'] = (pgbdt*0.804944+prf*0.802744)/(0.802744+0.804944)
test['prob_willsee'] = Y

In [6]:
pgbdt=Y

In [3]:
prf=Y

In [7]:
test.to_csv(data+"../result_09_16_02.csv", index=False)

In [6]:
Y

array([ 0.54248543,  0.53549656,  0.25285365, ...,  0.70869188,
        0.8427376 ,  0.22460439])

In [9]:
pt = pd.read_csv(data+"../libfm/test_result.csv", header=None, names=['prob_willsee'])

In [11]:
test['prob_willsee']=pt

In [16]:
(fm.predict_proba(test)>0.99).sum()

92698

In [20]:
_

array([ 0.,  0.,  0., ...,  0.,  0.,  0.])

In [30]:
train

<11112x11891 sparse matrix of type '<class 'numpy.float64'>'
	with 146128 stored elements in Compressed Sparse Row format>

In [31]:
test

<100015x11892 sparse matrix of type '<class 'numpy.float64'>'
	with 1315262 stored elements in Compressed Sparse Row format>

In [23]:
test[0,:].data

array([ 1.        ,  1.        ,  0.72826087,  0.15217391,  2.60869565,
        0.17366651,  0.27941176,  0.2254902 ,  0.00980392,  0.48529412,
        1.96568627,  0.18466436])

In [28]:
test[0,:].indptr

array([ 0, 12], dtype=int32)

In [25]:
train[0, :].data

array([ 1.        ,  1.        ,  0.39189189,  0.39189189,  0.04054054,
        0.17567568,  0.52027027,  0.97582781,  0.07478632,  0.70299145,
        0.04273504,  0.17948718,  0.6784188 ,  0.66357103])